In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv("./data/Mays - Arkusz1.csv")

df.head()

,ID,util,times30dpd,dti,bad
0,1,"13,30%",0,"27,00%",0
1,2,"84,20%",1,"44,00%",0
2,3,"31,30%",2,"38,00%",1
3,4,"1,90%",0,"2,00%",0
4,5,"15,50%",0,"36,00%",0


In [5]:
df["util"] = df["util"].str.replace("%", "").str.replace(",", ".").astype(float) / 100
df["dti"] = df["dti"].str.replace("%", "").str.replace(",", ".").astype(float) / 100

df.head()

,ID,util,times30dpd,dti,bad
0,1,0.133,0,0.27,0
1,2,0.842,1,0.44,0
2,3,0.313,2,0.38,1
3,4,0.019,0,0.02,0
4,5,0.155,0,0.36,0


In [36]:
df["utilscore"] = pd.cut(
    df["util"], bins=[0, 0.3, 0.5, 0.7, 1], labels=[35, 25, 10, 0], include_lowest=True
)

df["dpdscore"] = pd.cut(
    df["times30dpd"],
    bins=[0, 1, 2, 1000],
    right=False,
    labels=[50, 20, 0],
    include_lowest=True,
)

df["dtiscore"] = pd.cut(
    df["dti"], bins=[0, 0.2, 0.4, 1], labels=[75, 30, 0], include_lowest=True
)

df["score"] = (
    df["utilscore"].fillna(0).astype(int)
    + df["dpdscore"].fillna(0).astype(int)
    + df["dtiscore"].fillna(0).astype(int)
)

df.head(13)

,ID,util,times30dpd,dti,bad,utilscore,dpdscore,dtiscore,score
0,1,0.133,0,0.27,0,35,50,30,115
1,2,0.842,1,0.44,0,0,20,0,20
2,3,0.313,2,0.38,1,25,0,30,55
3,4,0.019,0,0.02,0,35,50,75,160
4,5,0.155,0,0.36,0,35,50,30,115
5,6,0.570,0,0.32,0,10,50,30,90
6,7,0.522,0,0.40,0,10,50,30,90
7,8,0.682,2,0.11,0,10,0,75,85
8,9,0.593,1,0.23,0,10,20,30,60
9,10,0.303,1,0.31,0,25,20,30,75


In [37]:
df["score"].mean()

np.float64(97.6805)

In [38]:
bad_counts = df[df["bad"] == 1].groupby("score").size()
total_counts = df.groupby("score").size()
relation = (bad_counts / total_counts).fillna(0)

relation

score
0      0.788372
10     0.711475
20     0.609302
25     0.657609
30     0.563046
35     0.541045
40     0.412162
45     0.504808
50     0.436567
55     0.353870
60     0.332527
65     0.281532
75     0.302824
80     0.211111
85     0.187822
90     0.133588
95     0.164384
100    0.116883
105    0.126344
110    0.105978
115    0.062453
120    0.084639
125    0.057325
130    0.051530
135    0.043541
150    0.025579
160    0.015133
dtype: float64

In [41]:
df["scoreclass"] = pd.cut(
    df["score"],
    bins=[0, 100, 1000],
    labels=["T", "F"],
    right=False,
    include_lowest=True,
)

In [42]:
scoreclass_bad_counts = df.groupby(["scoreclass", "bad"]).size().unstack(fill_value=0)
print(scoreclass_bad_counts)

bad            0     1
scoreclass            
T           6183  3442
F           9825   550


/var/folders/5j/c9443ny96w1dn8fvj92_g8700000gn/T/ipykernel_42915/2279798691.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  scoreclass_bad_counts = df.groupby(['scoreclass', 'bad']).size().unstack(fill_value=0)


In [43]:
# True Positives (TP) and False Negatives (FN)
TP = scoreclass_bad_counts.loc["T", 1]
FN = scoreclass_bad_counts.loc["F", 1]

# Calculate recall
recall = TP / (TP + FN)
recall

np.float64(0.8622244488977956)